In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import neurokit2 as nk

filename = 'prediction-Test.csv'
df = pd.read_csv(filename)
segment = df['ECG'].values
sr = 267  # sampling rate

cleaned = nk.ecg_clean(segment, sampling_rate=sr, method='neurokit')

rpeaks_info = nk.ecg_findpeaks(cleaned, sampling_rate=sr)
rpeaks = np.array(rpeaks_info['ECG_R_Peaks'])
_, waves = nk.ecg_delineate(cleaned, rpeaks, sampling_rate=sr, method='peak')

# Rentang waktu tampilan plot
start_time = 0  
end_time = 6    
start_idx = int(start_time * sr)
end_idx = int(end_time * sr)

time_axis = np.arange(len(cleaned)) / sr
plt.figure(figsize=(18, 7))
plt.plot(time_axis[start_idx:end_idx], cleaned[start_idx:end_idx], label="Sinyal EKG", color='black', linewidth=1.4, zorder=1)

colors = {
    "ECG_P_Onsets": 'crimson', "ECG_P_Peaks": 'dodgerblue', "ECG_P_Offsets": 'blue',
    "ECG_Q_Peaks": 'green', "ECG_R_Peaks": 'darkorange', "ECG_S_Peaks": 'purple',
    "ECG_T_Peaks": 'magenta'
}

labels = {
    "ECG_P_Onsets": "P Onsets",
    "ECG_P_Peaks": "Puncak P",
    "ECG_P_Offsets": "P Offsets",
    "ECG_Q_Peaks": "Puncak Q",
    "ECG_R_Peaks": "Puncak R",
    "ECG_S_Peaks": "Puncak S",
    "ECG_T_Peaks": "Puncak T"
}


for key in colors:
    if key in waves and waves[key] is not None:
        idxs = waves[key]
        idxs = [i for i in idxs if not np.isnan(i)]
        idxs_in_range = [i for i in idxs if start_idx <= i < end_idx]
        plt.scatter(
            np.array(idxs_in_range) / sr,
            cleaned[idxs_in_range],
            label=labels[key],
            marker='o',
            s=40,
            alpha=0.7,
            color=colors[key],
            zorder=5
        )

rpeaks_in_range = rpeaks[(rpeaks >= start_idx) & (rpeaks < end_idx)]
plt.scatter(
    rpeaks_in_range / sr,
    cleaned[rpeaks_in_range],
    label='Puncak R',
    color=colors["ECG_R_Peaks"],
    marker='o',
    s=40,
    alpha=0.7,
    zorder=5
)

p_onsets = waves.get("ECG_P_Onsets")
q_peaks  = waves.get("ECG_Q_Peaks")
s_peaks  = waves.get("ECG_S_Peaks")

durasi_p = []
durasi_qrs = []

valid_triplets = zip(p_onsets, q_peaks, s_peaks)
for i, (pon, q, s) in enumerate(valid_triplets):
    if not np.isnan(pon) and not np.isnan(q) and not np.isnan(s):
        pon, q, s = int(pon), int(q), int(s)
        if start_idx <= pon < end_idx and s < end_idx:
            dur_p = (q - pon) / sr
            dur_qrs = (s - q) / sr
            durasi_p.append(dur_p)
            durasi_qrs.append(dur_qrs)

            plt.hlines(
                y=cleaned[q] + 0.01,
                xmin=pon / sr,
                xmax=q / sr,
                color='saddlebrown',
                linestyle='--',
                label='Durasi P' if i == 0 else None,
                zorder=2
            )
            plt.text(
                x=(pon + q) / (2 * sr),
                y=cleaned[q] - 0.2,
                s=f"{dur_p*1000:.0f} ms",
                color='saddlebrown',
                ha='center',
                fontsize=8,
                zorder=3
            )

            plt.hlines(
                y=cleaned[s] + 0.01,
                xmin=q / sr,
                xmax=s / sr,
                color='darkslategrey',
                linestyle='--',
                label='Durasi QRS' if i == 0 else None,
                zorder=2
            )
            plt.text(
                x=(q + s) / (2 * sr),
                y=cleaned[s] - 0.2,
                s=f"{dur_qrs*1000:.0f} ms",
                color='darkslategrey',
                ha='center',
                fontsize=8,
                zorder=3
            )

x_minor_ticks = np.arange(start_time - 0.2, end_time + 0.2, 0.04)
x_major_ticks = np.arange(start_time - 0.2, end_time + 0.2, 0.2)
y_min = np.min(cleaned[start_idx:end_idx])
y_max = np.max(cleaned[start_idx:end_idx])
y_minor_ticks = np.arange(np.floor(y_min), np.ceil(y_max)+0.1, 0.1)
y_major_ticks = np.arange(np.floor(y_min), np.ceil(y_max)+1, 0.5)

for x in x_minor_ticks:
    plt.axvline(x=x, color='red', linestyle='-', linewidth=0.1, zorder=0)
for x in x_major_ticks:
    plt.axvline(x=x, color='red', linestyle='-', linewidth=0.4, zorder=0)

for y in y_minor_ticks:
    plt.axhline(y=y, color='red', linestyle='-', linewidth=0.1, zorder=0)
for y in y_major_ticks:
    plt.axhline(y=y, color='red', linestyle='-', linewidth=0.4, zorder=0)

plt.title(f"Anotasi P Onsets, P, Q, R, S, T, Durasi P, dan Durasi QRS pada Sinyal EKG yang Telah Dibersihkan")
plt.xlabel("Waktu (s)")
plt.ylabel("Amplitudo (mV)")
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()

p_onsets_all = waves.get("ECG_P_Onsets")
q_peaks_all  = waves.get("ECG_Q_Peaks")
s_peaks_all  = waves.get("ECG_S_Peaks")

durasi_p_all = []
durasi_qrs_all = []

min_len = min(len(p_onsets_all), len(q_peaks_all), len(s_peaks_all))
for i in range(min_len):
    pon = p_onsets_all[i]
    q = q_peaks_all[i]
    s = s_peaks_all[i]

    if not np.isnan(pon) and not np.isnan(q) and not np.isnan(s):
        dur_p = (q - pon) / sr
        dur_qrs = (s - q) / sr
        durasi_p_all.append(dur_p)
        durasi_qrs_all.append(dur_qrs)

print("\n=== Durasi P per beat (seluruh data) ===")
for i, d in enumerate(durasi_p_all):
    print(f"  Beat {i+1}: {d*1000:.2f} ms")

print("\n=== Durasi QRS per beat (seluruh data) ===")
for i, d in enumerate(durasi_qrs_all):
    print(f"  Beat {i+1}: {d*1000:.2f} ms")

print(f"\nRata-rata Durasi P: {np.mean(durasi_p_all)*1000:.2f} ms")
print(f"Rata-rata Durasi QRS: {np.mean(durasi_qrs_all)*1000:.2f} ms")